In [30]:
import os

old_folder = r'D:\Shiwei\20221023-hM1_hM1_Cx28_sample2\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Shiwei\20230406-hM1_CTP14_from_Cx28_sample2\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

os.path.isfile(before_position_file), os.path.isfile(after_position_file)



(True, True)

# Calculate rotation matrix

In [31]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [32]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)
R, T

(array([[ 0.99992858, -0.01195104],
        [ 0.01195104,  0.99992858]]),
 array([-898.73826546,  538.74439   ]))

In [18]:
np.arccos(R[0,1])

1.5712726301582611

# Translate 60x positions

In [34]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [35]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-2983.07326789 -2807.08457868]
 [-2985.52323178 -3012.06993834]
 [-2987.97319566 -3217.05529801]
 [-2990.42315955 -3422.04065767]
 [-2785.43779988 -3424.49062156]
 [-2782.987836   -3219.50526189]
 [-2780.53787211 -3014.51990223]
 [-2778.08790823 -2809.53454256]
 [-2775.63794435 -2604.5491829 ]
 [-2565.75265691 -2197.02842745]
 [-2568.2026208  -2402.01378712]
 [-2570.65258468 -2606.99914678]
 [-2573.10254856 -2811.98450645]
 [-2575.55251245 -3016.96986611]
 [-2578.00247633 -3221.95522578]
 [-2580.45244022 -3426.94058544]
 [-2582.9024041  -3631.92594511]
 [-2380.36700832 -3839.36126866]
 [-2377.91704444 -3634.37590899]
 [-2375.46708055 -3429.39054933]
 [-2373.01711667 -3224.40518966]
 [-2370.56715278 -3019.41983   ]
 [-2368.1171889  -2814.43447033]
 [-2365.66722502 -2609.44911067]
 [-2363.21726113 -2404.463751  ]
 [-2360.76729725 -2199.47839134]
 [-2358.31733336 -1994.49303167]
 [-2355.86736948 -1789.50767201]
 [-2148.43204593 -1586.97227623]
 [-2150.88200981 -1791.95763589]
 [-2153.33

In [36]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20230406-hM1_CTP14_from_Cx28_sample2\Alignment\translated_positions_all.txt


# Manual adjust positions

In [38]:
manual_positions_before_file = os.path.join(os.path.dirname(old_folder), '60x_positions_before.txt')
print(manual_positions_before_file)
manual_positions = np.loadtxt(manual_positions_before_file, delimiter=',')
print(manual_positions)

D:\Shiwei\20221023-hM1_hM1_Cx28_sample2\60x_positions_before.txt
[[ 2581.3  -2477.35]
 [ 1696.75 -4911.85]
 [-2598.55 -2838.25]
 [ 2299.75 -4481.65]]


In [39]:
translated_manual_positions = np.dot(manual_positions, R) + T
print('translated manual positions:')
print(np.round(translated_manual_positions, 1))

translated manual positions:
[[ 1652.8 -1969.3]
 [  739.2 -4393. ]
 [-3531.  -2268.2]
 [ 1347.3 -3970.1]]


In [40]:
manual_real_positions = [
    [1633.8, -1969.3],
    [715.6,-4393.5],
    [-3549.5,-2255.2],
    [1323.3,-3970.1],
]
manual_shifts = np.array(manual_real_positions) - translated_manual_positions[:len(manual_real_positions)]
manual_shifts

array([[-1.89704705e+01, -2.20849206e-02],
       [-2.35888268e+01, -4.67243211e-01],
       [-1.84772645e+01,  1.30475291e+01],
       [-2.39871016e+01, -3.00407996e-02]])

In [41]:
manual_shift = np.mean(manual_shifts, axis=0)
manual_shift

array([-21.25591585,   3.13204005])

In [42]:
adjusted_new_positions = new_positions + manual_shift

In [43]:
adjusted_new_positions[0], old_positions[0]

(array([-3004.32918375, -2803.95253863]), array([-2044.2, -3370.5]))

In [44]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20230406-hM1_CTP14_from_Cx28_sample2\Alignment\adjusted_translated_positions_all.txt
